In [2]:
# BBC Text Categorization
# https://www.kaggle.com/yufengdev/bbc-text-categorization/notebook
 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
 
import os
# print(os.listdir("../input"))
 
# Any results you write to the current directory are saved as output.
import itertools
import os
 
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
 
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
 
from tensorflow import keras
layers = keras.layers
models = keras.models
 
 
# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.6.0


In [ ]:
data = pd.read_csv("https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv")
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [3]:
data['category'].value_counts()

NameError: ignored

In [ ]:
data = data.sample(n=50, replace=False, random_state=127)
data.head()

,category,text
1522,business,fed chief warning on us deficit federal reserv...
1877,business,sa unveils more for all budget the south afr...
634,sport,mutu to hold talks with juventus disgraced for...
658,sport,kirwan demands italy consistency italy coach j...
919,politics,eu referendum could cost £80m it could cost £...


In [ ]:
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 40
Test size: 10


In [ ]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [ ]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [ ]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

In [ ]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [ ]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [ ]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (40, 1000)
x_test shape: (10, 1000)
y_train shape: (40, 5)
y_test shape: (10, 5)


In [ ]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 100
drop_ratio = 0.2

In [ ]:
# Build the model
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/100
2/2 [==============================] - 0s 62ms/step - loss: 1.7026 - accuracy: 0.2222 - val_loss: 1.1919 - val_accuracy: 0.7500
Epoch 2/100
2/2 [==============================] - 0s 19ms/step - loss: 0.6943 - accuracy: 0.8333 - val_loss: 0.9522 - val_accuracy: 0.5000
Epoch 3/100
2/2 [==============================] - 0s 22ms/step - loss: 0.3951 - accuracy: 0.8889 - val_loss: 0.8114 - val_accuracy: 0.7500
Epoch 4/100
2/2 [==============================] - 0s 21ms/step - loss: 0.2264 - accuracy: 1.0000 - val_loss: 0.7025 - val_accuracy: 0.7500
Epoch 5/100
2/2 [==============================] - 0s 18ms/step - loss: 0.1244 - accuracy: 1.0000 - val_loss: 0.6292 - val_accuracy: 0.7500
Epoch 6/100
2/2 [==============================] - 0s 18ms/step - loss: 0.0671 - accuracy: 1.0000 - val_loss: 0.5886 - val_accuracy: 0.7500
Epoch 7/100
2/2 [==============================] - 0s 20ms/step - loss: 0.0410 - accuracy: 1.0000 - val_loss: 0.5748 - val_accuracy: 0.7500
Epoch 8/100
2/2 [===

In [ ]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1/1 [==============================] - 0s 2ms/step - loss: 0.3358 - accuracy: 0.9000
Test loss: 0.33583250641822815
Test accuracy: 0.8999999761581421
